We have a tandem queue where customers leaving queue 1 can join queue 2 if it's not empty.

In [ ]:
!pip install simpy

In [ ]:
import simpy
import random

class MM1():
    def __init__(self, env, lambda_1, mu_1, mu_2):
        self.env = env
        self.queue_1 = simpy.Store(env)
        self.queue_2 = simpy.Store(env)
        self.lambda_1 = lambda_1
        self.mu_2 = mu_2
        self.mu_1 = mu_1
        self.data = {'arrival 1':[], 'arrival 2':[], 'departure 1':[], 'departure 2':[]}


    def arrive1(self):  #generate arrivals
        for _ in range(2*10**6):
            yield self.env.timeout(random.expovariate(self.lambda_1))
            self.queue_1.put('c1')
            self.data['arrival 1'].append(self.env.now)



    def service_1(self): #queue 1
        while True:
            yield self.queue_1.get()
            yield self.env.timeout(random.expovariate(self.mu_1))
            if len(self.queue_1.items) >= 0:
                self.queue_2.put('x')
                self.data['departure 1'].append(self.env.now)

    def service_2(self):#queue 2
        while True:
            yield self.queue_2.get()
            yield self.env.timeout(random.expovariate(self.mu_2))
            self.data['departure 2'].append(self.env.now)

In [ ]:
env = simpy.Environment()
lam, mu_1, mu_2 = 1, 2, 2
net = MM1(env, lam, mu_1, mu_2)
env.process(net.arrive1())

env.process(net.service_1())
env.process(net.service_2())

env.run()

l = 3*10**5
w1 = (sum(net.data['departure 2'][l:-l]) - sum(net.data['departure 1'][l:-l])) / len(net.data['departure 2'][l:-l])

print('waiting time is:',w1)


0.998599880719866
